In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import re

# increase display of columns in pandas
pd.set_option('display.max_colwidth', 200)
import os
import sys
sys.path.append('./dev1')
from evaluate import Evaluate

In [2]:
!pip install tk

In [13]:
import tkinter as tk

class QA(tk.Frame):
    
    def __init__(self,master):
        
        tk.Frame.__init__(self,master)
        self.master = master
        
        filepath = 'C:\\Users\\User\\NLP\\cleaned_df.csv'
        self.data = pd.read_csv(filepath)
        self.data.columns = [i.lower() for i in self.data.columns]
        self.cat_col = [col for col in self.data if col.startswith('category')][-1]
        cats = list(self.data[self.cat_col].unique())
        cats = sorted(cats)
        
        self.cat_selection = tk.StringVar()
        self.cat_selection.set(cats[0])
        
        self.cat_label = tk.Label(root,text="Select Category: ")
        self.cat_entry = tk.OptionMenu(root,self.cat_selection, *cats)
        
        self.submit = tk.Button(text="Submit Category",command=self.gen_question)
        
        self.cat_label.grid(row=0,column=0)
        self.cat_entry.grid(row=0,column=1)
        self.submit.grid(row=1,column=1)
        
    def gen_question(self):
        self.submit.destroy()

        self.category = self.cat_selection.get()
    
        mask_cat = (self.data[self.cat_col] == self.category)
        self.data_filter = self.data[mask_cat]
        self.data_filter = self.data_filter.sample(n=1)
    
        self.question = self.data_filter.iloc[0,2]
        self.ref_answer = self.data_filter.iloc[0,3]
            
        self.question_title = tk.Label(root,text=f'Question')
        self.question_label = tk.Label(root,text=self.question)
        
        self.answer_entry = tk.Entry(root)
        self.submit = tk.Button(text="Submit Answer",command=self.get_result)
        
        self.question_title.grid(row=5,column=0)
        self.question_label.grid(row=6,column=0)
        self.answer_entry.grid(row=7,column=0)
        self.submit.grid(row=7,column=1)            
                
    def get_result(self):
        self.answer = self.answer_entry.get()
        
        self.input = (
            self.question,
            self.ref_answer,
            self.answer
        )
        
        output = [
            Evaluate.is_invalid_case(self,self.input),
            Evaluate.is_match_case(self,self.input),
            Evaluate.is_ordering_case(self,self.input),
            Evaluate.is_aposthrope_case(self,self.input)         
        ]
        
        if output[0] is not None:
            self.output = output[0]
            
        if all(item is None for item in output):
            self.output = tuple(
                True,
                self.answer,
                None
            )
        if output[0] is None:
            self.output = [i for i in output if i is not None]
        
        root.destroy()

            
if __name__ == '__main__':
    root = tk.Tk()
    root.geometry("600x200+200+200")
    app = QA(root)
    root.mainloop()

    print(app.input)
    print(app.output)     

('In which city was the stamp officially released?', 'Boston', 'ss')
('False', 'ss', 'Invalid/not match answer')
